# --- Day 3: Crossed Wires ---
The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

What is the Manhattan distance from the central port to the closest intersection?

In [1]:
wire_paths = []

with open('../data/03a.txt', 'r') as f:
    for line in f:
        wire_paths.append(line.strip().split(','))

a_paths, b_paths = wire_paths

In [2]:
test0 = '''R8,U5,L5,D3
U7,R6,D4,L4'''

test1 = """R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83"""

test2 = """R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"""

In [3]:
def all_seen(paths):
    """Given a list of paths, create a dictionary of every point visited,
    and the fewest steps taken to get there.
    {(x, y): steps_taken}
    """
    a = (0, 0)
    steps = 0
    total_steps = {}
    seen = set()
    
    
    def update_dict(total_steps, a, steps):
        steps += 1
        if a not in total_steps:
            total_steps[a] = steps
        # `else` not needed,this can only have more steps than the initial assignment     
        return steps, total_steps
    
    
    for path in paths:
        a_start = a[:]    
        _dir, delta = path[0], int(path[1:])
        if _dir == 'R':
            while a[0] < a_start[0] + delta:
                a = (a[0] + 1, a[1])
                steps, total_steps = update_dict(total_steps, a, steps)
        if _dir == 'L':
            while a[0] > a_start[0] - delta:
                a = (a[0] - 1, a[1])
                steps, total_steps = update_dict(total_steps, a, steps)
        if _dir == 'U':
            while a[1] < a_start[1] + delta:
                a = (a[0], a[1] + 1)
                steps, total_steps = update_dict(total_steps, a, steps)
        if _dir == 'D':
            while a[1] > a_start[1] - delta:
                a = (a[0], a[1] - 1)
                steps, total_steps = update_dict(total_steps, a, steps)
    return total_steps


def shared_seen(a_paths=a_paths, b_paths=b_paths):
    a_set = set(all_seen(a_paths))
    b_set = set(all_seen(b_paths))
    return list(a_set.intersection(b_set))


def manhattan_min(shared):
    shared = [[int(x) for x in point] for point in shared]
    return sum(min(shared, key=lambda x: abs(x[0]) + abs(x[1])))


for test, ans in zip([test0, test1, test2], [6, 159, 135]):
    a_test, b_test = [x.split(',') for x in test.split('\n')]
    manhat_min = manhattan_min(shared_seen(a_test, b_test))
    assert manhat_min == ans, f'part A got {manhat_min}, should have been {ans}'


manhattan_min(shared_seen(a_paths, b_paths))  # 248

248

### --- Part Two ---
It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered.

What is the fewest combined steps the wires must take to reach an intersection?

In [4]:
a, b = all_seen(a_paths), all_seen(b_paths)

fewest_steps = min([a[point] + b[point] for point in shared_seen()])  
print(f'The fewest combined steps to get to a wire crossing is {fewest_steps}.')  # 28580

The fewest combined steps to get to a wire crossing is 28580.
